<a href="https://colab.research.google.com/github/jerryshenfewcher/Cits4012/blob/main/ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Abstract

Import data from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import json
import pandas as pd

path = '/content/drive/My Drive/ABSA/'
filenames = ['test.json', 'train.json', 'val.json']

# 读取JSON文件并转换为pandas DataFrame
datasets = {}
for filename in filenames:
    full_path = path + filename
    with open(full_path, 'r') as file:
        data = json.load(file)
        df = pd.json_normalize(data)  # 可以根据你的文件结构调整这里的代码
        datasets[filename.split('.')[0]] = df

# 现在 dataframes 列表中包含了三个DataFrame，分别对应三个JSON文件

for name, df in datasets.items():
  print(f"Dataset: {name}")
  print(df.info())
  print(df.head())

Dataset: test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   columns  1 non-null      object
 1   index    1 non-null      object
 2   data     1 non-null      object
dtypes: object(3)
memory usage: 152.0+ bytes
None
                        columns  \
0  [sentence, aspect, polarity]   

                                               index  \
0  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                data  
0  [[We went again and sat at the bar this time, ...  
Dataset: train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   columns  1 non-null      object
 1   index    1 non-null      object
 2   data     1 non-null      object
dtypes: object(3)
memory usage: 152.0+ bytes
None
       

 Introduction

Methods

 Experiments

Results

Conclusion